# Relatório Hands-On-5

Fernando Antonio Marques Schettini $^1$, Gabriel Mascarenhas Costa de Sousa$^2$, Jadson Nobre das Virgens$^2$

$^1$ Curso de Engenharia de Computação - Centro universitário SENAI CIMATEC, Salvador, Bahia, Brazil  

$^2$ Curso de Sistemas de Informação - Universidade do Estado da Bahia, Salvador, Bahia, Brazil

# Resumo


Este é o relátorio das atividades realizadas durante a execução da prática Hands-On-5 [1] . O relatório foi feito como atividade avaliativa da matéria Fundamentos de Programação Paralela, lecionada no centro universitário SENAI CIMATEC.

Dando continuidade às praticas de Hands-On, esse está dividido em três atividades relacionadas ao uso do modelo MPI[2] (Message Passing Interface), para o entendimento do conceito de paralelismo com memoria distribuída

# Introdução

Nesse Hands-On vamos trabalhar com três atividades que introduzem bem a ideia de distruibuição de tarefas entre processos.
Para isso, usaremos os conceitos de MPI ensinados em sala e através dos materiais de apoio compartilhados para:

1. Realizar diferentes operações matemáticas basicas em um mesmo vetor de inteiros
2. Solucionar equações algébricas de terceiro gral de forma dinâmica, onde cada variável pode ser inserida pelo usuário durante a execução do programa.
3. Realizar operações em uma matriz quadrada alterando sua diagonal principal, superdiagonal e subdiagonal, a partir de valores de uma variável "***K***" predefinidos.

Podemos ver então que temos já definidas as três operações que iremos aplicar: Soma, subtração e multiplicação.

A ideia aqui é que iremos dividir essas tarefas entre diferentes processos, onde um estará responsável por inicializar o vetor e delegar as tarefas aos outros três.

# Tarefa 1

A primeira atividade é bem simples, apenas para demonstrar a execução de um hello world utilizando as tecnicas misturadas de OpenMP com MPI. O código se deu da seguinte forma:

In [ ]:
%%writefile hybrid.c

#include <mpi.h>
#include <omp.h>
#include <stdio.h>
#include <stdlib.h>
int main( int argc, char *argv[])
{
  int nthreads,nprocs,idpro,idthr;
  int namelen;
  char processor_name[MPI_MAX_PROCESSOR_NAME];
  MPI_Init(&argc,&argv);
  MPI_Comm_size(MPI_COMM_WORLD,&nprocs);
  MPI_Comm_rank(MPI_COMM_WORLD,&idpro);
  MPI_Get_processor_name(processor_name,&namelen);
  #pragma omp parallel private(idthr) firstprivate(idpro,processor_name)
  {
    idthr = omp_get_thread_num();
    printf("Hello World thread %d,From %d processor %s\n",idthr,idpro,processor_name);
  }

  MPI_Finalize();
}

In [ ]:
%%shell
sudo apt-get install openmpi-bin openmpi-common libopenmpi-dev libgtk2.0-dev
sudo apt-get install libomp-dev gcc

In [ ]:
%%shell
mpicc hybrid.c -o hybryd -fopenmp
OMP_NUM_THREADS=8 mpirun --allow-run-as-root -np 2 ./hybryd

Como pudemos observar, foram abertos dois processos onde cada um deles abriu oito threads e todos, um por um, imprimiram na tela um "hello world" indicando seus respectivos números de thread/processo.

# Tarefa 2

A segunda atividade segue o mesmo principio de distribuição de tarefas, dessa vez para o calculo de uma função algebrica de terceiro gral, onde cada processo ficará encarregado de  calcular um pedaço da equação.

Começamos com os mesmos métodos de ***Init***, definição de ***size*** e de ***rank***. Lembrando sempre de incluir a lib mpi.h e declarar as demais variáveis necessárias para o programa.

In [ ]:
#include <stdio.h>
#include <math.h>
#include <mpi.h>

int main (int argc, char **argv){

  double coefficient[4], total, x, p[3], vars[3], result, w1[2], w2[2], w3[3];
  char c;
  int numOfProc, id, tag1 = 10, tag2 = 20, tag3 = 30;

  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &numOfProc);
  MPI_Comm_rank(MPI_COMM_WORLD, &id);
  MPI_Status status;
  ...

Como dessa vez, os processos irão receber valores diferentes com os quais trabalhar, também escolhemos criar tags diferentes para cada processo, apenas para ajudar na distinção dos comandos. Depois que fazemos essa parte inicial podemos continuar para a divisão de trabalho, sempre começando pelo rank do processo mestre.

Aqui o processo 0 começa imprimindo a fórmula da equação na tela e pedindo para que o usuário insira cada variavel relacionada a ela. Depois que todos os valores das variáveis são inseridos, os vetores com os dados que devem ser enviados a cada processo são separados e enviados um por um.

Por fim, o mestre aguarda o retorno dos valores já calculados por cada processo e os concatena no resultado final, o impimindo na tela.

In [ ]:
  ...
  MPI_Comm_rank(MPI_COMM_WORLD, &id);
  MPI_Status status;

  switch(id){
    case 0:
      printf ("\nf(x) = a*x^3 + b*x^2 + c*x + d\n");

      for(c = 'a'; c < 'e'; c++) {
        printf ("\nEnter the value of the 'constants' %c:\n", c);
        scanf ("%lf", &coefficient[c - 'a']);
      }

      printf("\nf(x) = %lf*x^3 + %lf*x^2 + %lf*x + %lf\n", coefficient[0], coefficient[1], coefficient[2], coefficient[3]);

      printf("\nEnter the value of 'x':\n");
      scanf("%lf", &x);
      
      w1[0] = coefficient[0];
      w2[0] = coefficient[1];
      w3[0] = coefficient[2];
      w3[1] = coefficient[3];
      w1[1] = w2[1] = w3[2] = x;
      
      MPI_Send(&w1, 2, MPI_DOUBLE, 1, tag1, MPI_COMM_WORLD);
      MPI_Send(&w2, 2, MPI_DOUBLE, 2, tag2, MPI_COMM_WORLD);
      MPI_Send(&w3, 3, MPI_DOUBLE, 3, tag3, MPI_COMM_WORLD);
      
      MPI_Recv(&p[0], 1, MPI_DOUBLE, 1, tag1, MPI_COMM_WORLD, &status);
      MPI_Recv(&p[1], 1, MPI_DOUBLE, 2, tag2, MPI_COMM_WORLD, &status);
      MPI_Recv(&p[2], 1, MPI_DOUBLE, 3, tag3, MPI_COMM_WORLD, &status);
      
      printf("\nf(%lf) = %lf\n", x, p[0]+p[1]+p[2]);
      break;
      ...

A atividade executada pelos processos filhos segue um esquema parecido com o da primeira tarefa, onde cada processo calcula uma parte independente da equação e a retorna para o processo mestre.

Ao final, também encerramos o MPI com o ***MPI_Finalize***.

In [ ]:
      ...  
      printf("\nf(%lf) = %lf\n", x, p[0]+p[1]+p[2]);
      break;
    
    case 1:
      printf("case 1");
      MPI_Recv(&vars, 2, MPI_DOUBLE, 0, tag1, MPI_COMM_WORLD, &status);
      //result = vars[0] * pow(vars[1], 3);
      result = vars[0] * vars[1] * vars[1] * vars[1];
      MPI_Send(&result, 1, MPI_DOUBLE, 0, tag1, MPI_COMM_WORLD);
      break;
    case 2:
      MPI_Recv(&vars, 2, MPI_DOUBLE, 0, tag2, MPI_COMM_WORLD, &status);
      //result = vars[0] * pow(vars[1], 2);
      result = vars[0] * vars[1] * vars[1];
      MPI_Send(&result, 1, MPI_DOUBLE, 0, tag2, MPI_COMM_WORLD);
      break;
    case 3:
      MPI_Recv(&vars, 3, MPI_DOUBLE, 0, tag3, MPI_COMM_WORLD, &status);
      result = vars[0] * vars[2] + vars[1];
      MPI_Send(&result, 1, MPI_DOUBLE, 0, tag3, MPI_COMM_WORLD);
      break;
  }

  MPI_Finalize();
  return 0;
}

# Tarefa 3

Por fim, a terceira atividade nos apresenta uma matriz quadrada onde os valores da diagonal principal, superdiagonal e subdiagonal devem sofrer alterações com base em valores de variaveis "***K***" pré-definidos. A solução dessa atividade se assemelha bastante à primeira parte desse Hands-On, pois enviaremos a mesma matriz inicializada pelo processo mestre, para cada um dos outros processos, apenas alterando a operação que será executada por cada um.

Começamos então com os mesmos métodos de anteriormente, além de aproveitar uma função de impressão de matriz já codificada pelo professor.

In [ ]:
#include <stdio.h>
#include <mpi.h>
#define ORDER 4

void printMatrix (int m[][ORDER]) {
  int i, j;
  for(i = 0; i < ORDER; i++) {
    printf ("| ");
    for (j = 0; j < ORDER; j++) {
      printf ("%3d ", m[i][j]);
    }
    printf ("|\n");
  }
  printf ("\n");
}

int main (int argc, char **argv){

  int k[3] = {100, 200, 300};
  int matrix[ORDER][ORDER], rcv1[ORDER][ORDER], rcv2[ORDER][ORDER], rcv3[ORDER][ORDER], size = ORDER*ORDER;
  int i, j, p, numProc, rank, tag = 10;
  
  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &numProc);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Status status;
  ...

Passamos então para o rank 0 que fica responsável por inicializar a matriz e a enviar para os outros processos, como de costume. Mas dessa vez, na hora da recepção, faremos atribuições distintas com base nas matrizes recebidas de cada processo.

Nesse caso, cada rank ficou responsável por uma diagonal, sendo elas a ***principal***, a ***superdiagonal*** e a ***subdiagonal***, da matriz quadrada. Com isso, a mensagem de resposta de cada processo contem uma matriz identica à original, com exceção da diagonal a qual o mesmo ficou encarregado.

Quando a resposta dos ranks filhos é recebida, o processo mestre roda um único loop passando por cada posição da matriz e apenas reatribui os novos valores às suas respectivas diagonais.

In [ ]:
  ...
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Status status;

  switch(rank){
    case 0:
      for(i = 0; i < ORDER; i++) {
        for(j = 0; j < ORDER; j++) {
          if( i == j )
            matrix[i][j] = i + j +1;
          else if(i == (j + 1)) {
            matrix[i][j] = i +  j + 1;
            matrix[j][i] = matrix[i][j];
          } else
           matrix[i][j] = 0;
        }
      }
      printMatrix(matrix);
      for(p = 1; p < numProc; p++){
        MPI_Send(&matrix, size, MPI_INT, p, tag, MPI_COMM_WORLD);
      }
      MPI_Recv(&rcv1, size, MPI_INT, 1, tag, MPI_COMM_WORLD, &status);
      MPI_Recv(&rcv2, size, MPI_INT, 2, tag, MPI_COMM_WORLD, &status);
      MPI_Recv(&rcv3, size, MPI_INT, 3, tag, MPI_COMM_WORLD, &status);
      for(i = 0; i < ORDER; i++){
        matrix[i][i] = rcv1[i][i];
        matrix[i + 1][i] = rcv2[i + 1][i];
        matrix[i][i + 1] = rcv3[i][i + 1];    
      }
      printMatrix(matrix);
      break;
      ...

Para finalizar, criamos um case ***default*** para os ranks filhos, ja que todos recebem a mesma matriz. Um único loop roda em cada processo e apenas nas posições designadas a eles, os mesmos executam a operação de soma com os respecivos valores de ***K***, retornando a matriz modificada para o processo mestre logo após isso.

Por fim, mais uma vez, encerramos o MPI.

In [ ]:
      ...
      printMatrix(matrix);
      break;

    default:
      MPI_Recv(&matrix, size, MPI_INT, 0, tag, MPI_COMM_WORLD, &status);

      for(i = 0; i < ORDER; i++){
        switch(rank){
          case 1:
            matrix[i][i] += k[0];
            break;
          case 2:
            matrix[i + 1][i] += k[1];
            break;
          case 3:
            matrix[i][i + 1] += k[2];
            break;
        }
      }
      MPI_Send(&matrix, size, MPI_INT, 0, tag, MPI_COMM_WORLD);
      break;
  }

  MPI_Finalize();
  return 0;
}

# Conclusões

Podemos notar claramente a semelhança entre as três atividades praticadas por esse Hands-On, o que ajuda bastante a se acostumar com os métodos e variáveis utilizados pelo MPI. Apesar de semelhantes, cada atividade exigiu uma estratégia diferente para o envio e recebimento das mensagens por cada processo, o que também auxilia no aprendizado de uso da ferramenta.

# Referencias

[1] M. Boratto. Hands-On Supercomputing with Parallel Computing. Available: https://github.com/
muriloboratto/Hands-On-Supercomputing-with-Parallel-Computing. 2022.

[2] Forum, Message Passing Interface. MPI: A Message-Passing Interface Standard. University of Tennessee,
1994, USA.

[3] B. Chapman, G. Jost and R. Pas. Using OpenMP: Portable Shared Memory Parallel Programming. The
MIT Press, 2007, USA.